# Defining Custom Display Logic for Your Own Objects

## Overview

In Python, objects can declare their textual representation using the `__repr__` method.  IPython expands on this idea and allows objects to declare other, richer representations including:

* HTML
* JSON
* PNG
* JPEG
* SVG
* LaTeX

This Notebook shows how you can add custom display logic to your own classes, so that they can be displayed using these rich representations. There are two ways of accomplishing this:

1. Implementing special display methods such as `_repr_html_`.
2. Registering a display function for a particular type.

In this Notebook we show how both approaches work.

Before we get started, we will import the various display functions for displaying the different formats we will create.

In [ ]:
from IPython.display import display
from IPython.display import (
    display_html, display_jpeg, display_png,
    display_javascript, display_svg, display_latex
)

## Implementing special display methods

The main idea of the first approach is that you have to implement special display methods, one for each representation you want to use. Here is a list of the names of the special methods and the values they must return:

* `_repr_html_`: return raw HTML as a string
* `_repr_json_`: return raw JSON as a string
* `_repr_jpeg_`: return raw JPEG data
* `_repr_png_`: return raw PNG data
* `_repr_svg_`: return raw SVG data as a string
* `_repr_latex_`: return LaTeX commands in a string surrounded by "$".

### Model Citizen: pandas

A prominent example of a package that has IPython-aware rich representations of its objects is [pandas](http://pandas.pydata.org/).

A pandas DataFrame has a rich HTML table representation,
using `_repr_html_`.


In [ ]:
import io
import pandas

In [ ]:
%%writefile data.csv
Date,Open,High,Low,Close,Volume,Adj Close
2012-06-01,569.16,590.00,548.50,584.00,14077000,581.50
2012-05-01,584.90,596.76,522.18,577.73,18827900,575.26
2012-04-02,601.83,644.00,555.00,583.98,28759100,581.48
2012-03-01,548.17,621.45,516.22,599.55,26486000,596.99
2012-02-01,458.41,547.61,453.98,542.44,22001000,540.12
2012-01-03,409.40,458.24,409.00,456.48,12949100,454.53


In [ ]:
df = pandas.read_csv("data.csv")
pandas.set_option('display.notebook_repr_html', False)
df

rich HTML can be activated via `pandas.set_option`.

In [ ]:
pandas.set_option('display.notebook_repr_html', True)
df

In [ ]:
lines = df._repr_html_().splitlines()
print("\n".join(lines[:20]))

### Exercise

Write a simple `Circle` Python class.  Don't even worry about properties such as radius, position, colors, etc. To help you out use the following representations (remember to wrap them in Python strings):

For HTML:

    &#x25CB;

For SVG:

    <svg width="100px" height="100px">
        <circle cx="50" cy="50" r="20" stroke="black" stroke-width="1" fill="white"/>
    </svg>

For LaTeX (wrap with `$` and use a raw Python string):

    \bigcirc

After you write the class, create an instance and then use `display_html`, `display_svg` and `display_latex` to display those representations.

Tips : you can slightly tweek the representation to know from which `_repr_*_` method it came from. 
For example in my solution the svg representation is blue, and the HTML one show "`HTML`" between brackets.

### Solution

Here is my simple `MyCircle` class:

In [ ]:
# %load ../../exercises/IPython Kernel/soln/mycircle.py
class MyCircle(object):

    def __init__(self, center=(0.0,0.0), radius=1.0, color='blue'):
        self.center = center
        self.radius = radius
        self.color = color

    def _repr_html_(self):
        return "&#x25CB; (<b>html</b>)"

    def _repr_svg_(self):
        return """<svg width="100px" height="100px">
           <circle cx="50" cy="50" r="20" stroke="black" stroke-width="1" fill="blue"/>
        </svg>"""
    
    def _repr_latex_(self):
        return r"$\bigcirc \LaTeX$"

    def _repr_javascript_(self):
        return "alert('I am a circle!');"


Now create an instance and use the display methods:

In [ ]:
c = MyCircle()

In [ ]:
display_html(c)

In [ ]:
display_svg(c)

In [ ]:
display_latex(c)

In [ ]:
display_javascript(c)

## Adding IPython display support to existing objects

When you are directly writing your own classes, you can adapt them for display in IPython by following the above example.  But in practice, we often need to work with existing code we can't modify.  We now illustrate how to add these kinds of extended display capabilities to existing objects. To continue with our example above, we will add a PNG representation to our `Circle` class using Matplotlib.

### Model citizen: sympy

[SymPy](http://sympy.org) is another model citizen that defines rich representations of its object.
Unlike pandas above, sympy registers display formatters via IPython's display formatter API, rather than declaring `_repr_mime_` methods.

In [ ]:
from sympy import Rational, pi, exp, I, symbols
x, y, z = symbols("x y z")

In [ ]:
r = Rational(3,2)*pi + exp(I*x) / (x**2 + y)
r

SymPy provides an `init_printing` function that sets up advanced $\LaTeX$
representations of its objects.

In [ ]:
from sympy.interactive.printing import init_printing
init_printing()
r

To add a display method to an existing class, we must use IPython's display formatter API.  Here we show all of the available formatters:

In [ ]:
ip = get_ipython()
for mime, formatter in ip.display_formatter.formatters.items():
    print('%24s : %s' % (mime, formatter.__class__.__name__))


Let's grab the PNG formatter:

In [ ]:
png_f = ip.display_formatter.formatters['image/png']

We will use the `for_type` method to register our display function.

In [ ]:
png_f.for_type?

As the docstring describes, we need to define a function the takes the object as a parameter and returns the raw PNG data.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
class AnotherCircle(object):
    def __init__(self, radius=1, center=(0,0), color='r'):
        self.radius = radius
        self.center = center
        self.color = color
    
    def __repr__(self):
        return "<%s Circle with r=%s at %s>" % (
                    self.color,
                    self.radius,
                    self.center,
                )
    
c = AnotherCircle()
c

In [ ]:
from IPython.core.pylabtools import print_figure

def png_circle(circle):
    """Render AnotherCircle to png data using matplotlib"""
    fig, ax = plt.subplots()
    patch = plt.Circle(circle.center,
                       radius=circle.radius,
                       fc=circle.color,
                       )
    ax.add_patch(patch)
    plt.axis('scaled')
    data = print_figure(fig, 'png')
    # We MUST close the figure, otherwise IPython's display machinery
    # will pick it up and send it as output, resulting in a double display
    plt.close(fig)
    return data

In [ ]:
c = AnotherCircle()
print(repr(png_circle(c)[:10]))

Now we register the display function for the type:

In [ ]:
png_f.for_type(AnotherCircle, png_circle)

Now all `Circle` instances have PNG representations!

In [ ]:
c2 = AnotherCircle(radius=2, center=(1,0), color='g')
c2

In [ ]:
display_png(c2)

## return the object

In [ ]:
# for demonstration purpose, I do the same with a circle that has no _repr_javascript method
class MyNoJSCircle(MyCircle):
    
    def _repr_javascript_(self):
        return

cNoJS = MyNoJSCircle()

Of course you can now still return the object, and this will use compute all the representations, store them in the notebook and show you the appropriate one.

In [ ]:
cNoJS

Or just use `display(object)` if you are in a middle of a loop

In [ ]:
for i in range(3):
    display(cNoJS)

Advantage of using `display()` versus `display_*()` is that all representation will be stored in the notebook document and notebook file, they are then availlable for other frontends or post-processing tool like `nbconvert`.

Let's compare `display()` vs `display_html()` for our circle in the Notebook Web-app and we'll see later the difference in nbconvert.

In [ ]:
print("I should see a nice html circle in web-app, but")
print("nothing if the format I'm viewing the notebook in")
print("does not support html")
display_html(cNoJS)

In [ ]:
print("Whatever the format I will see a representation")
print("of my circle")
display(cNoJS)

In [ ]:
print("Same if I return the object")
cNoJS

In [ ]:
print("But not if I print it")
print(cNoJS)

## Cleanup

In [ ]:
!rm -f data.csv